# LoRA低秩适配学习笔记

## 一、了解LoRA（Low-Rank Adaptation）

### （一）LoRA的定义： 
LoRA是一种参数高效的微调方法，通过在预训练矩阵的权重中引入低秩分解来实现参数高效的微调。  
LoRA的核心思想是：大型预训练模型的权重更新有低秩特性，因此可以用低秩矩阵来近似表示权重变化。

### （二）LoRA的基本原理：  
**核心假设:** 预训练模型在特定任务上的权重更新可以近似为低秩矩阵，即权重更新矩阵$\Delta W$可以分解为两个小矩阵的乘积。  
$$\Delta W=B\times A$$   
其中:  
$A\in R^{r×k}$  
$B\in R^{d×r}$  
$ r≪min(d,k)$ r是秩参数。  
**在标准线性变换中：**  
$$h=W_0x$$
**使用LoRA后：**
$$h=(W_0+\Delta W)x=(W_0+B\times A)x=W_0x+BAx$$  
其中$W_0$是冻结的预训练权重矩阵，B和A是可训练的低秩矩阵，$\Delta W$是权重更新。

### （三）LoRA工作流程：   
1. 冻结预训练权重：保持原始权重矩阵$W_0$不变  
2. 引入低秩矩阵：添加可训练的低秩矩阵$B$和$A $ 
3. 前向传播：在前向传播时计算$W_0+B\times A$  
4. 参数更新：只优化$B$和$A$矩阵  

### （四）为什么LoRA有效：
**理论基础：**  
1、低秩假设的合理性：研究表明，预训练模型在微调过程中权重的更新往往具有低秩特性，这意味着大部分更新信息可以被少数几个主要方向捕获。  
2、神经网络的过参数化：大型神经网络通常具有过参数化的特性，许多参数之间存在冗余，因此可以用更少的参数来近似表示权重更新。  

**有效性原理分析：**  
1、保持预训练知识：原始权重矩阵保持不变，避免灾难性遗忘问题，保留了预训练模型的通用功能。  
2、高效的参数利用：低秩矩阵能够捕获主要的更新方向，减少了参数冗余，提升了参数利用效率。  
3、良好的泛化能力：低秩约束有了正则化的作用，避免过拟合，提升模型的泛化能力。

## 二、LoRA矩阵的初始化方法和原理：  
### （一）对矩阵B和A的初始化： 

矩阵B使用0矩阵初始化 **（升维矩阵）**  $B=0$  
矩阵A使用高斯分布初始化 **（降维矩阵）**  $A=N~(0,1)$   
$A=N~(0,\frac{\sigma^2}{r})$ （其中$\sigma^2$是标准方差通常为1，r是秩，这么做的原因是 **控制低秩更新的整体尺度，使其不依赖于r的大小**，这其实是将原本要$\Delta W=\frac{\alpha}{r}BA$的那个缩放参数吸收进了初始化，使外面得以保持$\Delta W=BA$）

### （二）对初始化的考虑：   

**对B矩阵要初始化为0的原因：**  
1. 初始无影响：初始时$B\times A=0$不影响原始矩阵。
2. 渐进学习：模型可以从0开始逐步学习适配。
3. 稳定性：避免了初始损失和梯度剧烈震荡。

**为什么不能对A矩阵进行初始化为0：**   
1. 梯度消失：初始梯度为0，无法有效更新。
2. 对称性问题：所有神经元初始相同，无法有效分化。
3. 学习困难：模型难以从0开始学习有效表示。

**为什么对A矩阵使用高斯初始化：**
1. 打破对称性：不同的初始值帮助神经元学习不同的特征。  
2. 提供初始信号：为后续层提供非0的输入信息。  
3. 梯度流动：确保反向传播时有有效的梯度流。  

**为什么二者不可以同时为0：**    
用运算分析，见教案  

### （三）变换初始化方法的后果考虑  

**升维矩阵用高斯初始化，降维矩阵用0初始化：**   
1. 初始影响：即使降维矩阵为零，升维矩阵的非零值仍可能产生影响。  
2. 梯度问题：降维矩阵为零可能导致梯度流动不畅。   
3. 学习效率：可能需要更长时间才能学习到有效的适配。

**都使用0初始化：**  
1. 梯度消失：初始时所有梯度为零，无法开始学习。  
2. 对称性问题：所有参数初始相同，无法有效分化。  
3. 学习失败：模型可能无法从零开始学习。

**都使用高斯初始化：**  
1. 初始干扰：LoRA在初始时就会对模型产生显著影响。  
2. 稳定性问题：可能导致训练初期的不稳定。  
3. 预训练知识破坏：可能干扰预训练模型的已有知识。  

## 三、为什么LoRA使用两个矩阵相乘

### （一）低秩分解的必要性  
使用两个矩阵相乘的原因在于效率，假设原始维度为$d\times k$:  
一个矩阵需要参数：$dk$  
两个矩阵需要参数：$dr+rk$ （r远小于d,k）  
参数量显著减少

### （二）不能只要一个矩阵的原因  
1、参数效率的损失：如果只用一个矩阵，就无法实现参数的显著减少。    
2、表达能力的限制：单个矩阵可能无法有效捕获权重更新的主要方向。  
3、低秩约束：两个矩阵的乘积天然具有低秩特性，符合LoRA的核心假设。  

### （三）低秩近似的有效性  
根据矩阵分析理论，许多实际矩阵都可以用低秩近似来有效表示，特别是权重更新矩阵。  
在多个NLP任务中，LoRA使用较小的rank值（如4、8、16）就能达到很好的效果，证明了低秩近似的有效性。

## 四、LoRA微调的层的选择

### 注意力层LoRA  
作用：  
1、控制注意力机制的行为  
2、调整模型关注的信息  
3、影响序列中不同位置的交互  

### 前馈网络层LoRA  
作用：  
1、控制特征变换过程    
2、调整非线性变换行为  
3、影响模型的表达能力  

## 五、LoRA中Rank和Alpha参数    
Alpha参数：alpha 是 LoRA 的“控制旋钮”，用来调节低秩更新（ΔW）对原始模型的影响强度。 

### （一）Rank对微调效果的影响  
1、过小：表达能力不足，可能欠拟合。  
2、过大：参数过多，可能过拟合。    
3、适中：平衡表达能力和泛化性能。  


### （二）Alpha对微调效果的影响  
1、过小：适配强度不足，效果不明显。  
2、过大：过度改变原模型，可能破坏预训练知识。  
3、适中：有效适配，保持预训练优势。  

## 六、LoRA的变体  
### （一）LoRA+  
**实现原理：**  
矩阵B和A的梯度范数（梯度向量的欧几里得长度，越大越陡）有显著差异，可以设置不同的学习率。  
具体而言：  
A的梯度范数通常较小  
B的梯度范数通常较大  
因此：  
对矩阵A使用高学习率$\mu_A=\alpha_A\times \mu_0$  
对矩阵B使用低学习率$\mu_B=\alpha_B\times \mu_0$  
其中$\mu_0$是初始学习率，$\alpha_A$是矩阵A的学习率倍数  
这样设置的原因：  
矩阵A需要更快的学习速度来快速适应任务  
矩阵B需要较慢的学习速度来保持稳定性  

### （二）AdaLoRA(Adaptive LoRA) 
**核心思想：**  
通过奇异值分解（SVD）来动态调整LoRA的秩，实现更精细的参数控制，根据参数重要性自动调整每一个LoRA模块的秩。  
**优化点：**  
1、动态秩调整：根据重要性动态调整每个LoRA模块的秩。  
2、参数重要性评估：通过奇异值评估参数重要性。   
3、自适应剪枝：移除不重要的参数。  

**数学基础：**  
**奇异值分解**  
定义：奇异值分解（SVD）就是将一个任意的实数或复数矩阵分解为三个特定矩阵的乘积。  
具体做法，见教案
**奇异值对于矩阵的重要性：**  
奇异值反映了矩阵在各个主方向上的能量或重要性。较大的奇异值对应更重要的信息，较小的奇异值对应次要信息。  
**重要性评估原理：**  
奇异值的平方$\sigma_i^2$代表了该方向上的方差或信息量。    
累积奇异值的平方和代表了矩阵的总能量$\sum_{r=i}^r \sigma_i^2$  
通过保留主要的奇异值，可以近似表示原矩阵的主要信息。  

**实现原理：**  
在训练过程中定期进行SVD分析，根据奇异值的大小决定保留哪些参数。  
具体步骤：  
1、SVD分解：对LoRA矩阵的BA矩阵进行奇异值分解  
$$W_{lora}=U\Sigma V^T=\sum_{i=1}^r\sigma_i u_i v_i^T$$
其中，$\sigma_i$是降序排列后的第i个奇异值，$u_i$是左奇异向量$v_i$是右奇异向量。   
2、重要性评估：根据奇异值大小评估参数重要性  
$$Importance_i=\sigma^2$$  
3、秩调整：保留重要性高的奇异值对应的参数(保留前K个重要性大于阈值参数$\epsilon$的方向)   
$$
r_{new}=arg\min\limits_k \frac{\sum_{i=k+1}^r \sigma_i^2}{\sum_{i=1}^r \sigma_i^2}≤\epsilon
$$  
其中，$\epsilon$是阈值参数，表示可以接受的信息损失比例。  
4、参数更新：重构LoRA矩阵继续训练  

**自适应调整策略：**  
AdaLoRA采用以下策略来决定何时该**增加**还是**减少**秩  
1、性能监控：监控训练损失和验证损失，当验证损失停滞或增加时考虑调整秩。  
2、重要性阈值：设定重要性阈值$\epsilon$m，移除重要性低于阈值的奇异值对应的参数。  
3、最小最大秩限制：防止秩降为0或无限增长。  
4、控制调整频率：不是每一次迭代都调整，每固定步数调整一次。

**优势：**     
1、参数效率提升：自动移除不重要参数   
2、性能保持：保持模型性能的同时减少参数   
3、自适应性：根据任务需求自动调整  
4、理论基础扎实：基于SVD的数学原理，有坚实的理论支撑  

**实际应用的消耗考虑：**  
1. SVD计算开销：SVD计算相对耗时，需要平衡计算开销和收益  
2. 调整频率：不是每次迭代都进行调整，通常每隔一定步数进行一次  
3. 最小秩限制：设置最小秩限制，避免秩降为0  
4. 稳定性：确保调整过程不会破坏训练稳定性  

### （三）QLoRA  
原理：在 LoRA 基础上引入4-bit 量化（如 NF4）和分页优化器（Paged Optimizers）。  
改进：将预训练模型权重量化为 4-bit 存储，仅在前向传播时反量化为 16-bit；LoRA 适配器仍以 16-bit 训练。  
效果：大幅降低显存占用（可实现 7B 模型在 24GB 显卡上微调），使大模型微调可在消费级设备上运行  

### （四）DoRA  
DoRA 不是直接学习权重更新，而是将预训练权重分解为“方向”和“幅度”两部分，然后只用低秩方式去调整“方向”，从而更高效地引导模型适应新任务  


**DoRA分解原理：**  
不同于传统LoRA，DoRA分解为幅度和方向  
$$
W=(m+\Delta m)\cdot \frac{v+\Delta v}{||v+\Delta v||}
$$  
其中，原始权重是$W_0$，$v=W_0$是方向参数，$m=||v||$是其范数（幅度参数）      
$\Delta m$是一个可学习的低秩偏移  
$\Delta v$是对应的变量，且v会分解为低秩更新矩阵$\Delta v=B_vA_v$

**效果：**  
更精细地控制权重更新，解耦了方向与尺度变化，显著提升模型理解和泛化能力，尤其在复杂任务上优于标准 LoRA。